# Solve the Market Split problem with Kipu Quantum's Iskay Quantum Optimizer

{/* cspell:ignore adiabaticity, HUBO, bitflip, metaheuristic, fontweight, fontsize, QOBLIB, Zuse, Kochenberger, Tramontani, Weninger, edgecolor */}

<Admonition type="note" title="Note">
Qiskit Functions are an experimental feature available only to IBM Quantum&reg; Premium Plan, Flex Plan, and On-Prem (via IBM Quantum Platform API) Plan users. They are in preview release status and subject to change.
</Admonition>

*Usage estimate: 20 seconds on a Heron r2 processor. (NOTE: This is an estimate only. Your runtime might vary.)*

## Background

This tutorial demonstrates how to solve the Market Split problem using [Kipu Quantum's Iskay quantum optimizer](/docs/guides/kipu-optimization) [\[1\]](#references). The Market Split problem represents a real-world resource allocation challenge where markets must be partitioned into balanced sales regions to meet exact demand targets.

### The Market Split challenge

The Market Split problem presents a deceptively simple yet computationally formidable challenge in resource allocation. Consider a company with $m$ products being sold across $n$ different markets, where each market purchases a specific bundle of products (represented by the columns of matrix $A$). The business objective is to partition these markets into two balanced sales regions such that each region receives exactly half the total demand for every product.

**Mathematical formulation:**

We seek a binary assignment vector $x$, where:
- $x_j = 1$ assigns market $j$ to Region A
- $x_j = 0$ assigns market $j$ to Region B
- The constraint $Ax = b$ must be satisfied, where $b$ represents the target sales (typically half the total demand per product)

**Cost function:**

To solve this problem, we minimize the squared constraint violation:

$$C(x) = ||Ax - b||^2 = \sum_{i=1}^{m} \left(\sum_{j=1}^{n} A_{ij}x_j - b_i\right)^2$$

where:
- $A_{ij}$ represents the sales of product $i$ in market $j$
- $x_j \in \{0,1\}$ is the binary assignment of market $j$
- $b_i$ is the target sales for product $i$ in each region
- The cost equals zero precisely when all constraints are satisfied

Each term in the sum represents the squared deviation from the target sales for a particular product. When we expand this cost function, we get:

$$C(x) = x^T A^T A x - 2b^T A x + b^T b$$

Since $b^T b$ is a constant, minimizing $C(x)$ is equivalent to minimizing the quadratic function $x^T A^T A x - 2b^T A x$, which is exactly a QUBO (Quadratic Unconstrained Binary Optimization) problem.

**Computational complexity:**

Despite its straightforward business interpretation, this problem exhibits remarkable computational intractability:
- **Small-scale failure**: Conventional Mixed Integer Programming solvers fail on instances with as few as seven products under a timeout of one hour [\[4\]](#references)
- **Exponential growth**: The solution space grows exponentially ($2^n$ possible assignments), making brute force approaches infeasible

This severe computational barrier, combined with its practical relevance to territory planning and resource allocation, makes the Market Split problem an ideal benchmark for quantum optimization algorithms [\[4\]](#references).

### What makes Iskay's approach unique?

The Iskay optimizer uses the **bf-DCQO (bias-field digitized counterdiabatic quantum optimization)** algorithm [\[1\]](#references), which represents a significant advancement in quantum optimization:

**Circuit efficiency**: The bf-DCQO algorithm achieves remarkable gate reduction [\[1\]](#references):
- Up to **10 times fewer entangling gates** than Digital Quantum Annealing (DQA)
- Significantly shallower circuits enable:
  - Less error accumulation during quantum execution
  - Ability to tackle larger problems on current quantum hardware
  - No need for error mitigation techniques

**Non-variational design**: Unlike variational algorithms requiring approximately 100 iterations, bf-DCQO typically needs only **approximately 10 iterations** [\[1\]](#references). This is achieved through:
- Intelligent bias-field calculations from measured state distributions
- Starting each iteration from an energy state near the previous solution
- Integrated classical post-processing with local search

**Counterdiabatic protocols**: The algorithm incorporates counterdiabatic terms that suppress unwanted quantum excitations during short evolution times, enabling the system to remain near the ground state even with rapid transitions [\[1\]](#references).

## Requirements

Before starting this tutorial, ensure that you have the following installed:

* Qiskit IBM Runtime (`pip install qiskit-ibm-runtime`)
* Qiskit Functions (`pip install qiskit-ibm-catalog`)
* NumPy (`pip install numpy`)
* Requests (`pip install requests`)

You will also need to get access to the Iskay Quantum Optimizer function from the Qiskit Functions Catalog.

## Setup

First, import all required packages for this tutorial.

In [ ]:
import os
import tempfile
import time
from typing import Tuple, Optional

import numpy as np
import requests

from qiskit_ibm_catalog import QiskitFunctionsCatalog

from qiskit_addon_opt_mapper import OptimizationProblem
from qiskit_addon_opt_mapper.converters import OptimizationProblemToQubo

print("All required libraries imported successfully")

### Configure IBM Quantum Credentials

Define your [IBM Quantum Platform](https://quantum.cloud.ibm.com/) credentials. You will need:
- **API Token**: Your 44-character API key from IBM Quantum Platform
- **Instance CRN**: Your IBM Quantum instance identifier (format: `hub/group/project`)

In [ ]:
token = "<YOUR_API_KEY>"
instance = "<YOUR_INSTANCE_CRN>"

## Step 1: Map problem to quantum circuits and operators

We begin by mapping our classical problem to a quantum-compatible representation. This step involves:

1. Connecting to the Iskay Quantum Optimizer
2. Loading and formulating the Market Split problem
3. Understanding the bf-DCQO algorithm that will solve it

### Connect to Iskay Quantum Optimizer

We begin by establishing a connection to the Qiskit Functions Catalog and loading the Iskay quantum optimizer. The Iskay optimizer is a quantum function provided by Kipu Quantum that implements the bf-DCQO algorithm for solving optimization problems on quantum hardware.

In [ ]:
catalog = QiskitFunctionsCatalog(token=token)
iskay_solver = catalog.load("kipu-quantum/iskay-quantum-optimizer")

print("Iskay optimizer loaded successfully")
print("Ready to solve optimization problems using bf-DCQO algorithm")

### Load and Formulate the Problem

#### Understanding the Problem Data Format

Problem instances from QOBLIB (Quantum Optimization Benchmarking Library) [2] are stored in a simple text format. Let's examine the actual content of our target instance `ms_03_200_177.dat`:

```text
3 20
60   92  161   53   97    2   75   81    6  139  132   45  108  112  181   93  152  200  164   51 1002
176  196   41  143    2   88    0   79   10   71   75  148   82  135   34  187   33  155   58   46  879
68   68  179  173  127  163   48   49   99   78   44   52  173  131   73  198   84  109  180   95 1040
```

**Format Structure:**
- **First line:** `3 20`
  - `3` = number of products (constraints/rows in matrix $A$)
  - `20` = number of markets (variables/columns in matrix $A$)

- **Next 3 lines:** Coefficient matrix $A$ and target vector $b$
  - Each line has 21 numbers: first 20 are row coefficients, last is the target
  - Line 2: `60 92 161 ... 51 | 1002`
    - First 20 numbers: How much of Product 1 each of the 20 markets sells
    - Last number (1002): Target sales for Product 1 in one region
  - Line 3: `176 196 41 ... 46 | 879`
    - Product 2 sales per market and target (879)
  - Line 4: `68 68 179 ... 95 | 1040`
    - Product 3 sales per market and target (1040)

**Business Interpretation:**
- Market 0 sells: 60 units of Product 1, 176 units of Product 2, 68 units of Product 3
- Market 1 sells: 92 units of Product 1, 196 units of Product 2, 68 units of Product 3
- And so on for all 20 markets...
- **Goal**: Split these 20 markets into two regions where each region gets exactly 1002 units of Product 1, 879 units of Product 2, and 1040 units of Product 3

#### QUBO Transformation

## From Constraints to QUBO: The Mathematical Transformation

The power of quantum optimization lies in transforming constrained problems into unconstrained quadratic forms [4]. For the Market Split problem, we convert the equality constraints

$$ Ax = b $$

where $x ∈ \{0,1\}^n$, into a QUBO by penalizing constraint violations using the **qiskit_addon_opt_mapper** package.

**The Penalty Method:**
We'll use the `OptimizationProblem` class to define our constrained problem, then convert it to QUBO format using `OptimizationProblemToQubo`. This automatically handles the penalty-based transformation.

### Implementing Data Loading and QUBO Conversion Functions

We now define three utility functions:
1. `parse_marketsplit_dat()` - Parses the `.dat` file format and extracts matrices $A$ and $b$
2. `fetch_marketsplit_data()` - Downloads problem instances directly from the QOBLIB repository
3. `marketsplit_to_qubo()` - Converts the problem into QUBO format suitable for Iskay

In [20]:
def parse_marketsplit_dat(filename: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Parse a market split problem from a .dat file format.

    Parameters
    ----------
    filename : str
        Path to the .dat file containing the market split problem data.

    Returns
    -------
    A : np.ndarray
        Coefficient matrix of shape (m, n) where m is the number of products
        and n is the number of markets.
    b : np.ndarray
        Target vector of shape (m,) containing the target sales per product.
    """
    with open(filename, "r", encoding="utf-8") as f:
        lines = [
            line.strip()
            for line in f
            if line.strip() and not line.startswith("#")
        ]

    if not lines:
        raise ValueError("Empty or invalid .dat file")

    # First line: m n (number of products and markets)
    m, n = map(int, lines[0].split())

    # Next m lines: each row of A followed by corresponding element of b
    A, b = [], []
    for i in range(1, m + 1):
        values = list(map(int, lines[i].split()))
        A.append(values[:-1])  # First n values: product sales per market
        b.append(values[-1])  # Last value: target sales for this product

    return np.array(A, dtype=np.int32), np.array(b, dtype=np.int32)


def fetch_marketsplit_data(
    instance_name: str = "ms_03_200_177.dat",
) -> Tuple[Optional[np.ndarray], Optional[np.ndarray]]:
    """
    Fetch market split data directly from the QOBLIB repository.

    Parameters
    ----------
    instance_name : str
        Name of the .dat file to fetch (default: "ms_03_200_177.dat").

    Returns
    -------
    A : np.ndarray or None
        Coefficient matrix if successful, None if failed.
    b : np.ndarray or None
        Target vector if successful, None if failed.
    """
    url = f"https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library/-/raw/main/01-marketsplit/instances/{instance_name}"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        with tempfile.NamedTemporaryFile(
            mode="w", suffix=".dat", delete=False, encoding="utf-8"
        ) as f:
            f.write(response.text)
            temp_path = f.name

        try:
            return parse_marketsplit_dat(temp_path)
        finally:
            os.unlink(temp_path)
    except Exception as e:
        print(f"Error: {e}")
        return None, None

### Loading the Problem Instance

Now we load the specific problem instance `ms_03_200_177.dat` from QOBLIB [2]. This instance has:
- 3 products (constraints)
- 20 markets (binary decision variables)
- Over 1 million possible market assignments to explore ($2^{20} = 1,048,576$)

In [21]:
# Load the problem instance
instance_name = "ms_03_200_177.dat"
A, b = fetch_marketsplit_data(instance_name=instance_name)

if A is not None:
    print("Successfully loaded problem instance from QOBLIB")
    print("\nProblem Instance Analysis:")
    print("=" * 50)
    print(f"Coefficient Matrix A: {A.shape[0]} × {A.shape[1]}")
    print(f"   → {A.shape[0]} products (constraints)")
    print(f"   → {A.shape[1]} markets (decision variables)")
    print(f"Target Vector b: {b}")
    print("   → Target sales per product for each region")
    print(
        f"Solution Space: 2^{A.shape[1]} = {2**A.shape[1]:,} possible assignments"
    )

Successfully loaded problem instance from QOBLIB

Problem Instance Analysis:
Coefficient Matrix A: 3 × 20
   → 3 products (constraints)
   → 20 markets (decision variables)
Target Vector b: [1002  879 1040]
   → Target sales per product for each region
Solution Space: 2^20 = 1,048,576 possible assignments


### Converting to QUBO Format

We now transform the constrained optimization problem into QUBO format:

In [ ]:
# Create optimization problem
ms = OptimizationProblem(instance_name.replace(".dat", ""))

# Add binary variables (one for each market)
ms.binary_var_list(A.shape[1])

# Add equality constraints (one for each product)
for idx, rhs in enumerate(b):
    ms.linear_constraint(A[idx, :], sense="==", rhs=rhs)

# Convert to QUBO with penalty parameter
qubo = OptimizationProblemToQubo(penalty=1).convert(ms)

print("QUBO Conversion Complete:")
print("=" * 50)
print(f"Number of variables: {qubo.get_num_vars()}")
print(f"Constant term: {qubo.objective.constant}")
print(f"Linear terms: {len(qubo.objective.linear.to_dict())}")
print(f"Quadratic terms: {len(qubo.objective.quadratic.to_dict())}")

### Converting QUBO to Iskay Format

Now we need to convert the QUBO object into the dictionary format required by Kipu Quantum's Iskay optimizer. The Iskay format uses string keys with specific patterns:
- `"()"` for the constant term
- `"(i, )"` for linear coefficients (note the space before the closing parenthesis)
- `"(i, j)"` for quadratic coefficients

In [ ]:
# Convert QUBO to Iskay dictionary format
iskay_input_problem = {"()": qubo.objective.constant}
iskay_input_problem.update(
    {
        f"({idx}, )": float(val)
        for idx, val in enumerate(qubo.objective.linear)
    }
)
iskay_input_problem.update(
    {
        str(key): float(val)
        for key, val in qubo.objective.quadratic.to_dict().items()
    }
)

print("Iskay Dictionary Format:")
print("=" * 50)
print(f"Total coefficients: {len(iskay_input_problem)}")
print(f"  • Constant term: {iskay_input_problem['()']}")
print(
    f"  • Linear terms: {sum(1 for k in iskay_input_problem.keys() if k != '()' and ', )' in k)}"
)
print(
    f"  • Quadratic terms: {sum(1 for k in iskay_input_problem.keys() if k != '()' and ', )' not in k)}"
)
print("\nSample coefficients:")

# Get first 10 and last 5 items properly
items = list(iskay_input_problem.items())
first_10 = list(enumerate(items[:10]))
last_5 = list(enumerate(items[-5:], start=len(items) - 5))

for i, (key, value) in first_10 + last_5:
    coeff_type = (
        "constant"
        if key == "()"
        else "linear"
        if ", )" in key
        else "quadratic"
    )
    print(f"  {key}: {value} ({coeff_type})")
print("  ...")
print("\n✓ Problem ready for Iskay optimizer!")

### Understanding the bf-DCQO Algorithm

Before we run the optimization, let's understand the sophisticated quantum algorithm that powers Iskay: **bf-DCQO (bias-field digitized counterdiabatic quantum optimization)** [1].

#### What is bf-DCQO?

bf-DCQO is based on the time evolution of a quantum system where the problem solution is encoded in the **ground state** (lowest energy state) of the final quantum Hamiltonian [1]. The algorithm addresses a fundamental challenge in quantum optimization:

**The Challenge**: Traditional adiabatic quantum computing requires very slow evolution to maintain ground state conditions according to the adiabatic theorem. This demands increasingly deep quantum circuits as problem complexity grows, leading to more gate operations and accumulated errors.

**The Solution**: bf-DCQO uses counterdiabatic protocols to enable rapid evolution while maintaining ground state fidelity, dramatically reducing circuit depth.

#### Mathematical Framework

The algorithm minimizes a cost function of the form:

$$\min_{(x_1,x_2,...,x_n) \in D} C(x_1,x_2,...,x_n)$$

where $D = \{0,1\}^n$ for binary variables and:

$$C(x) = a + \sum_i b_i x_i + \sum_{i,j} c_{ij} x_i x_j + ... + \sum g_{k_1,...,k_m} x_{k_1}...x_{k_m}$$

For our Market Split problem, the cost function is:

$$C(x) = ||Ax - b||^2 = x^T A^T A x - 2 b^T A x + b^T b$$

#### The Role of Counterdiabatic Terms

**Counterdiabatic terms** are additional terms introduced into the time-dependent Hamiltonian that suppress unwanted excitations during the quantum evolution. Here's why they're crucial:

In adiabatic quantum optimization, we evolve the system according to a time-dependent Hamiltonian:

$$H(t) = \left(1 - \frac{t}{T}\right) H_{\text{initial}} + \frac{t}{T} H_{\text{problem}}$$

where $H_{\text{problem}}$ encodes our optimization problem. To maintain the ground state during rapid evolution, we add counterdiabatic terms:

$$H_{\text{CD}}(t) = H(t) + H_{\text{counter}}(t)$$

These counterdiabatic terms:
1. **Suppress unwanted transitions**: Prevent the quantum state from jumping to excited states during fast evolution
2. **Enable shorter evolution times**: Allow us to reach the final state much faster without violating adiabaticity
3. **Reduce circuit depth**: Shorter evolution leads to fewer gates and less error

The practical impact is dramatic: bf-DCQO uses up to **10 times fewer entangling gates** than Digital Quantum Annealing [1], making it practical for today's noisy quantum hardware.

#### Bias-Field Iterative Optimization

Unlike variational algorithms that optimize circuit parameters through many iterations, bf-DCQO uses a **bias-field guided approach** that converges in approximately 10 iterations [1]:

**Iteration Process:**

1. **Initial Quantum Evolution**: Start with a quantum circuit implementing the counterdiabatic evolution protocol

2. **Measurement**: Measure the quantum state to obtain a probability distribution over bitstrings

3. **Bias-Field Calculation**: Analyze the measurement statistics and calculate an optimal bias-field $h_i$ for each qubit:
   $$h_i = \text{f}(\text{measurement statistics}, \text{previous solutions})$$

4. **Next Iteration**: The bias-field modifies the Hamiltonian for the next iteration:
   $$H_{\text{next}} = H_{\text{problem}} + \sum_i h_i \sigma_i^z$$

   This allows starting near the previously found good solution, effectively performing a form of "quantum local search"

5. **Convergence**: Repeat until the solution quality stabilizes or a maximum number of iterations is reached

**Key Advantage**: Each iteration provides meaningful progress toward the optimal solution by incorporating information from previous measurements, unlike variational methods that must explore the parameter space blindly.

#### Integrated Classical Post-Processing

After the quantum optimization converges, Iskay performs classical **local search** post-processing:

- **Bit-Flip Exploration**: Systematically or randomly flip bits in the best measured solution
- **Energy Evaluation**: Calculate $C(x)$ for each modified solution
- **Greedy Selection**: Accept improvements that lower the cost function
- **Multiple Passes**: Perform several passes (controlled by `postprocessing_level`)

This hybrid approach compensates for bit-flip errors from hardware imperfections and readout errors, ensuring high-quality solutions even on noisy quantum devices.

#### Why bf-DCQO Excels on Current Hardware

The bf-DCQO algorithm is specifically designed to excel on today's noisy intermediate-scale quantum (NISQ) devices [1]:

1. **Error Resilience**: Fewer gates (10 times reduction) means dramatically less error accumulation
2. **No Error Mitigation Required**: The algorithm's inherent efficiency eliminates the need for expensive error mitigation techniques [1]
3. **Scalability**: Can handle problems with up to 156 qubits (156 binary variables) with direct qubit mapping [1]
4. **Proven Performance**: Achieves 100% approximation ratios on benchmark MaxCut and HUBO instances [1]

Now let's see this powerful algorithm in action on our Market Split problem!

## Step 2: Optimize for target hardware

The bf-DCQO algorithm automatically handles circuit optimization, creating shallow quantum circuits with counterdiabatic terms specifically designed for the target backend.

### Configure the Optimization

The Iskay optimizer requires several key parameters to effectively solve your optimization problem. Let's examine each parameter and its role in the quantum optimization process:

#### Required Parameters

| Parameter | Type | Description | Example |
|-----------|------|-------------|---------|
| **problem** | `Dict[str, float]` | QUBO coefficients in string-key format | `{"()": -21.0, "(0,4)": 0.5, "(0,1)": 0.5}` |
| **problem_type** | `str` | Format specification: `"binary"` for QUBO or `"spin"` for Ising | `"binary"` |
| **backend_name** | `str` | Target quantum device | `"ibm_fez"` |

#### Essential Concepts

- **Problem Format**: We use `"binary"` since our variables are binary (0/1), representing market assignments.
- **Backend Selection**: Choose between the available QPUs (e.g., `"ibm_fez"`) based on your needs and compute resource instance.
- **QUBO Structure**: Our problem dictionary contains the exact coefficients from the mathematical transformation.

#### Advanced Options (Optional)

Iskay provides fine-tuning capabilities through optional parameters. While the defaults work well for most problems, you can customize the behavior for specific requirements:

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| **shots** | `int` | 10000 | Quantum measurements per iteration (higher = more accurate) |
| **num_iterations** | `int` | 10 | Algorithm iterations (more iterations can improve solution quality) |
| **use_session** | `bool` | True | Use IBM sessions for reduced queue times |
| **seed_transpiler** | `int` | None | Set for reproducible quantum circuit compilation |
| **direct_qubit_mapping** | `bool` | False | Map virtual qubits directly to physical qubits |
| **job_tags** | `List[str]` | None | Custom tags for job tracking |
| **preprocessing_level** | `int` | 0 | Problem preprocessing intensity (0-3) - see details below |
| **postprocessing_level** | `int` | 2 | Solution refinement level (0-2) - see details below |
| **transpilation_level** | `int` | 0 | Transpiler optimization trials (0-5) - see details below |
| **transpile_only** | `bool` | False | Analyze circuit optimization without running full execution |

**Preprocessing Levels (0-3)**: Specially important for larger problems that cannot currently fit on the coherence times of the hardware. While higher preprocessing levels achieve shallower circuit depths by approximations in the problem transpilation:
- **Level 0**: Exact, longer circuits
- **Level 1**: Good balance between accuracy and approximation, cutting out only the gates with angles in the lowest 10 percentile
- **Level 2**: Slightly higher approximation, cutting out the gates with angles in the lowest 20 percentile and using `approximation_degree=0.95` in the transpilation
- **Level 3**: Maximum approximation level, cutting out the gates in the lowest 30 percentile and using `approximation_degree=0.90` in the transpilation

**Transpilation Levels (0-5)**: Control the advanced transpiler optimization trials for quantum circuit compilation. This can lead to an increase in classical overhead, and for some cases it might not change the circuit depth. The default value `2` in general leads to the smallest circuit and is relatively fast.
- **Level 0**: Optimization of the decomposed DCQO circuit (layout, routing, scheduling)
- **Level 1**: Optimization of PauliEvolutionGate and then the decomposed DCQO circuit (max_trials=10)
- **Level 2**: Optimization of PauliEvolutionGate and then the decomposed DCQO circuit (max_trials=15)
- **Level 3**: Optimization of PauliEvolutionGate and then the decomposed DCQO circuit (max_trials=20)
- **Level 4**: Optimization of PauliEvolutionGate and then the decomposed DCQO circuit (max_trials=25)
- **Level 5**: Optimization of PauliEvolutionGate and then the decomposed DCQO circuit (max_trials=50)

**Postprocessing Levels (0-2)**: Control how much classical optimization, compensating for bitflip errors with different number of greedy passes of a local search:
- **Level 0**: 1 pass
- **Level 1**: 2 passes
- **Level 2**: 3 passes

**Transpile-Only Mode**: Now available for users who want to analyze circuit optimization without running the full quantum algorithm execution.

#### Custom Configuration Example

Here's how you might configure Iskay with different settings:
```python
custom_options = {
    "shots": 15_000,                                        # Higher shot count for better statistics
    "num_iterations": 12,                                   # More iterations for solution refinement
    "preprocessing_level": 1,                               # Light preprocessing for problem simplification
    "postprocessing_level": 2,                              # Maximum postprocessing for solution quality
    "transpilation_level": 3,                               # Using higher transpilation level for circuit optimization
    "seed_transpiler": 42,                                  # Fixed seed for reproducible results
    "job_tags": ["market_split"]                            # Custom tracking tags
}
```

For this tutorial, we will keep most of the default parameters and will only change the number of bias-field iterations:

In [ ]:
# Specify the target backend
backend_name = "ibm_fez"

# Set the number of bias-field iterations and set a tag to identify the jobs
options = {
    "num_iterations": 3,  # Change number of bias-field iterations
    "job_tags": ["market_split_example"],  # Tag to identify jobs
}

# Configure Iskay optimizer
iskay_input_problem = {
    "problem": iskay_input_problem,
    "problem_type": "binary",
    "backend_name": backend_name,
    "instance": instance,
    "token": token,
    "options": options,
}

print("Iskay Optimizer Configuration:")
print("=" * 40)
print(f"  Backend: {backend_name}")
print(f"  Problem: {len(iskay_input_problem)} QUBO coefficients")
print("  Algorithm: bf-DCQO")

## Step 3: Execute on target hardware

We now submit our problem to run on IBM Quantum hardware. The bf-DCQO algorithm will:
1. Construct shallow quantum circuits with counterdiabatic terms
2. Execute approximately 10 iterations with bias-field optimization
3. Perform classical post-processing with local search
4. Return the optimal market assignment

In [ ]:
# Submit the optimization job
print("Submitting optimization job to Kipu Quantum...")
print(
    f"Problem size: {A.shape[1]} variables, {len(iskay_input_problem)} QUBO terms"
)
print(
    "Algorithm: bf-DCQO (bias-field digitized counterdiabatic quantum optimization)"
)

job = iskay_solver.run(**iskay_input_problem)

print("\nJob successfully submitted!")
print(f"Job ID: {job.job_id}")
print("Optimization in progress...")
print(
    f"The bf-DCQO algorithm will efficiently explore {2**A.shape[1]:,} possible assignments"
)

### Monitoring Job Status

You can check the current status of your optimization job. The possible statuses are:
- `QUEUED`: Job is waiting in the queue
- `RUNNING`: Job is currently executing on quantum hardware
- `DONE`: Job completed successfully
- `CANCELED`: Job was canceled
- `ERROR`: Job encountered an error

In [ ]:
# Check job status
print(f"Job status: {job.status()}")

### Waiting for Completion

This cell will block until the job completes. The optimization process includes:
- Queue time (waiting for quantum hardware access)
- Execution time (running the bf-DCQO algorithm with approximately 10 iterations)
- Post-processing time (classical local search)

Typical completion times range from a few minutes to tens of minutes depending on queue conditions.

In [ ]:
# Wait for job completion
while True:
    status = job.status()
    print(
        f"Waiting for job {job.job_id} to complete... (status: {status})",
        end="\r",
        flush=True,
    )
    if status in ["DONE", "CANCELED", "ERROR"]:
        print(
            f"\nJob {job.job_id} completed with status: {status}" + " " * 20
        )
        break
    time.sleep(30)

# Retrieve the optimization results
result = job.result()
print("\nOptimization complete!")

## Step 4: Post-process results

We now post-process the quantum execution results. This includes:
- Analyzing the solution structure
- Validating constraint satisfaction
- Benchmarking against classical approaches

### Analyze Results

#### Understanding the Result Structure

Iskay returns a comprehensive result dictionary containing:
- **`solution`**: A dictionary mapping variable indices to their optimal values (0 or 1)
- **`solution_info`**: Detailed information including:
  - `bitstring`: The optimal assignment as a binary string
  - `cost`: The objective function value (should be 0 for perfect constraint satisfaction)
  - `mapping`: How bitstring positions map to problem variables
  - `seed_transpiler`: Seed used for reproducibility
- **`prob_type`**: Whether the solution is in binary or spin format

Let's examine the solution returned by the quantum optimizer.

In [ ]:
# Display the optimization results
print("Optimization Results")
print("=" * 50)
print(f"Problem Type: {result['prob_type']}")
print("\nSolution Info:")
print(f"  Bitstring: {result['solution_info']['bitstring']}")
print(f"  Cost: {result['solution_info']['cost']}")
print(f"Total QPU time: {result['solution_info']['qpu_time']} seconds")
print("\nSolution (first 10 variables):")
for i, (var, val) in enumerate(list(result["solution"].items())[:10]):
    print(f"  {var}: {val}")
print("  ...")

#### Solution Validation

Now we validate whether the quantum solution satisfies the Market Split constraints. The validation process checks:

**What is a Constraint Violation?**
- For each product $i$, we calculate the actual sales in Region A: $(Ax)_i$
- We compare this to the target sales $b_i$
- The **violation** is the absolute difference: $|(Ax)_i - b_i|$
- A **feasible solution** has zero violations for all products

**What We Expect:**
- **Ideal case**: Total violation = 0 (all constraints perfectly satisfied)
  - Region A gets exactly 1002 units of Product 1, 879 units of Product 2, and 1040 units of Product 3
  - Region B gets the remaining units (also 1002, 879, and 1040 respectively)
- **Good case**: Total violation is small (near-optimal solution)
- **Poor case**: Large violations indicate the solution doesn't satisfy the business requirements

The validation function will compute:
1. Actual sales per product in each region
2. Constraint violations for each product
3. Market distribution between regions

In [ ]:
def validate_solution(A, b, solution):
    """Validate market split solution."""
    x = np.array(solution)
    region_a = A @ x
    region_b = A @ (1 - x)
    violations = np.abs(region_a - b)

    return {
        "target": b,
        "region_a": region_a,
        "region_b": region_b,
        "violations": violations,
        "total_violation": np.sum(violations),
        "is_feasible": np.sum(violations) == 0,
        "region_a_markets": int(np.sum(x)),
        "region_b_markets": len(x) - int(np.sum(x)),
    }


# Convert bitstring to list of integers and validate
optimal_assignment = [
    int(bit) for bit in result["solution_info"]["bitstring"]
]
validation = validate_solution(A, b, optimal_assignment)

#### Interpreting the Validation Results

The validation results show whether the quantum optimizer found a feasible solution. Let's examine:

**Feasibility Check:**
- **`is_feasible = True`** means the solution perfectly satisfies all constraints (total violation = 0)
- **`is_feasible = False`** means some constraints are violated

**Sales Analysis:**
- Compare Target vs Actual sales for each product
- For a perfect solution: Actual = Target for all products in both regions
- The difference indicates how close we are to the desired market split

**Market Distribution:**
- Shows how many markets are assigned to each region
- There's no requirement for equal numbers of markets, only that sales targets are met

In [ ]:
print("Solution Validation")
print("=" * 50)
print(f"Feasible solution: {validation['is_feasible']}")
print(f"Total constraint violation: {validation['total_violation']}")

print("\nSales Analysis (Target vs Actual):")
for i, (target, actual_a, actual_b) in enumerate(
    zip(validation["target"], validation["region_a"], validation["region_b"])
):
    violation_a = abs(actual_a - target)
    violation_b = abs(actual_b - target)
    print(f"  Product {i+1}:")
    print(f"    Target: {target}")
    print(f"    Region A: {actual_a} (violation: {violation_a})")
    print(f"    Region B: {actual_b} (violation: {violation_b})")

print("\nMarket Distribution:")
print(f"  Region A: {validation['region_a_markets']} markets")
print(f"  Region B: {validation['region_b_markets']} markets")

#### Solution Quality Assessment

Based on the validation results above, we can assess the quality of the quantum solution:

**If `is_feasible = True` (Total violation = 0):**
- The quantum optimizer successfully found an optimal solution
- All business constraints are perfectly satisfied
- This demonstrates quantum advantage on a problem where classical solvers struggle [4]

**If `is_feasible = False` (Total violation > 0):**
- The solution is near-optimal but not perfect
- Small violations may be acceptable in practice
- Consider adjusting optimizer parameters:
  - Increase `num_iterations` for more optimization passes
  - Increase `postprocessing_level` for more classical refinement
  - Increase `shots` for better measurement statistics

**Cost Function Interpretation:**
- The `cost` value from `solution_info` equals $||Ax - b||^2$
- Cost = 0 indicates perfect constraint satisfaction
- Higher cost values indicate larger constraint violations

### Benchmark Against Classical Approaches

#### Why Compare with Classical Algorithms?

To fully appreciate the value of quantum optimization, it's instructive to compare the bf-DCQO results against classical baseline methods. This comparison helps us understand:

1. **Solution Quality**: Does quantum optimization find better solutions than classical methods?
2. **Computational Efficiency**: How do execution times compare for this problem size?
3. **Scalability Insights**: Which approaches remain viable as problem complexity grows?

We'll implement a simple comparison framework to evaluate our quantum solution against three classical baselines:

- **Enhanced Simulated Annealing**: A sophisticated metaheuristic with adaptive temperature scheduling and restart mechanisms
- **Random Search**: A simple stochastic baseline that randomly samples the solution space
- **Brute Force Search**: An exhaustive enumeration providing a guaranteed optimal solution (feasible only for small problems)

#### Comparison Framework

First, we define a simple comparison framework that standardizes how we evaluate and record results from different algorithms.

In [ ]:
import matplotlib.pyplot as plt


class SimpleComparator:
    """Framework for comparing optimization algorithms on the Market Split problem."""

    def __init__(self, A, b):
        """
        Initialize the comparator with problem data.

        Parameters
        ----------
        A : np.ndarray
            Coefficient matrix (products × markets)
        b : np.ndarray
            Target sales vector
        """
        self.A = A
        self.b = b
        self.n = A.shape[1]
        self.results = []

    def objective_function(self, x):
        """
        Calculate constraint violations: ||Ax - b||²

        Parameters
        ----------
        x : array-like
            Binary solution vector

        Returns
        -------
        float
            Squared constraint violation (0 = perfect solution)
        """
        return np.sum((self.A @ x - self.b) ** 2)

    def add_result(self, name, solution, exec_time):
        """
        Add an algorithm's result to the comparison.

        Parameters
        ----------
        name : str
            Algorithm name
        solution : list
            Binary solution vector
        exec_time : float
            Execution time in seconds
        """
        x = np.array(solution)
        obj_value = self.objective_function(x)
        is_feasible = obj_value == 0

        self.results.append(
            {
                "Algorithm": name,
                "Solution": solution,
                "Objective": obj_value,
                "Feasible": is_feasible,
                "Time": exec_time,
            }
        )

        print(
            f"{name}: Objective={obj_value}, Feasible={is_feasible}, Time={exec_time:.3f}s"
        )


# Initialize comparator with our problem instance
comparator = SimpleComparator(A, b)
print("Comparison framework initialized successfully")

#### Baseline 1: Iskay Quantum Optimizer (bf-DCQO)

First, we record the quantum optimization result we obtained earlier. Note that the execution time should reflect the actual time from job submission to result retrieval (including queue time, quantum execution, and post-processing).

In [ ]:
comparator.add_result(
    name="Quantum (bf-DCQO)",
    solution=optimal_assignment,
    exec_time=result["solution_info"]["qpu_time"],
)

#### Baseline 2: Enhanced Simulated Annealing

Simulated Annealing is a probabilistic metaheuristic inspired by the physical process of annealing in metallurgy. Our implementation includes several enhancements:

**Key Features:**
- **Adaptive Temperature Schedule**: Different cooling rates for exploration (early phase) and exploitation (late phase)
- **Variable Neighborhood**: Switches between single-bit and multi-bit flips based on optimization progress
- **Restart Mechanism**: Escapes local minima by restarting from random solutions after prolonged stagnation
- **Acceptance Probability**: Accepts worse solutions with probability $e^{-\Delta/T}$ where $\Delta$ is the cost increase and $T$ is temperature

**Parameters:**
- Maximum iterations: 20,000,000
- Initial temperature: 50,000 (high for exploration)
- Stagnation limit: 500,000 iterations before restart
- Maximum restarts: 20

In [ ]:
def enhanced_simulated_annealing(A, b, max_iter=20_000_000, seed=42):
    """
    Enhanced Simulated Annealing with adaptive cooling and restart mechanisms.

    Parameters
    ----------
    A : np.ndarray
        Coefficient matrix
    b : np.ndarray
        Target vector
    max_iter : int
        Maximum iterations (default: 20,000,000)
    seed : int
        Random seed for reproducibility

    Returns
    -------
    tuple
        (best_solution, execution_time)
    """
    start_time = time.time()
    np.random.seed(seed)

    # Initialize with random solution
    current = np.random.randint(0, 2, A.shape[1])
    current_cost = np.sum((A @ current - b) ** 2)

    best = current.copy()
    best_cost = current_cost

    # Temperature schedule parameters
    initial_temp = 50000.0

    # Stagnation tracking
    last_improvement = 0
    stagnation_limit = 500_000
    restart_count = 0
    max_restarts = 20

    # Statistics
    accepted_moves = 0
    total_moves = 0

    for i in range(max_iter):
        # Adaptive neighborhood based on progress
        neighbor = current.copy()
        progress = i / max_iter

        if progress < 0.3:  # Exploration phase
            if np.random.random() < 0.7:
                flip_idx = np.random.randint(A.shape[1])
                neighbor[flip_idx] = 1 - neighbor[flip_idx]
            else:
                num_flips = np.random.choice([2, 3, 4], p=[0.5, 0.3, 0.2])
                flip_indices = np.random.choice(
                    A.shape[1], size=num_flips, replace=False
                )
                for idx in flip_indices:
                    neighbor[idx] = 1 - neighbor[idx]
        else:  # Exploitation phase
            if np.random.random() < 0.9:
                flip_idx = np.random.randint(A.shape[1])
                neighbor[flip_idx] = 1 - neighbor[flip_idx]
            else:
                flip_indices = np.random.choice(
                    A.shape[1], size=2, replace=False
                )
                for idx in flip_indices:
                    neighbor[idx] = 1 - neighbor[idx]

        neighbor_cost = np.sum((A @ neighbor - b) ** 2)
        total_moves += 1

        # Adaptive temperature schedule
        if progress < 0.5:
            temperature = initial_temp * np.exp(-3 * progress)
        else:
            temperature = (
                initial_temp
                * np.exp(-3 * 0.5)
                * np.exp(-5 * (progress - 0.5))
            )

        # Acceptance criterion
        accepted = False
        if neighbor_cost < current_cost:
            current = neighbor
            current_cost = neighbor_cost
            accepted = True
        elif temperature > 0:
            delta = neighbor_cost - current_cost
            acceptance_prob = np.exp(-delta / temperature)
            if np.random.random() < acceptance_prob:
                current = neighbor
                current_cost = neighbor_cost
                accepted = True

        if accepted:
            accepted_moves += 1

        # Update best solution
        if current_cost < best_cost:
            best = current.copy()
            best_cost = current_cost
            last_improvement = i

        # Restart mechanism
        if (
            i - last_improvement > stagnation_limit
            and restart_count < max_restarts
            and best_cost > 0
            and i > stagnation_limit
        ):
            acceptance_rate = accepted_moves / max(total_moves, 1)
            print(
                f"  Restart {restart_count + 1}: No improvement for {stagnation_limit:,} iterations (acceptance rate: {acceptance_rate:.3f})"
            )

            current = np.random.randint(0, 2, A.shape[1])
            current_cost = np.sum((A @ current - b) ** 2)
            last_improvement = i
            restart_count += 1
            initial_temp *= 1.8

            accepted_moves = 0
            total_moves = 0

        # Early termination
        if best_cost == 0:
            print(f"  Perfect solution found at iteration {i:,}!")
            break
        elif time.time() - start_time > 300:
            print(f"  Timeout reached after {i:,} iterations")
            break

        # Progress reporting
        if i % 500_000 == 0 and i > 0:
            acceptance_rate = accepted_moves / max(total_moves, 1)
            print(
                f"  Iteration {i:,}: Best cost = {best_cost}, Current cost = {current_cost}, Temp = {temperature:.3f}, Accept rate = {acceptance_rate:.3f}"
            )
            accepted_moves = 0
            total_moves = 0

    final_acceptance_rate = (
        accepted_moves / max(total_moves, 1) if total_moves > 0 else 0
    )
    print(
        f"  Final: Best cost = {best_cost} after {restart_count} restarts (final acceptance rate: {final_acceptance_rate:.3f})"
    )

    return best.tolist(), time.time() - start_time


# Run Enhanced Simulated Annealing
print("Running Enhanced Simulated Annealing...")
sa_solution, sa_time = enhanced_simulated_annealing(A, b)
comparator.add_result("Enhanced Simulated Annealing", sa_solution, sa_time)

#### Baseline 3: Random Search

Random Search provides a simple stochastic baseline by uniformly sampling the solution space. While naive, it serves as an important reference point:

**How It Works:**
- Randomly generate binary solutions
- Evaluate each solution's cost
- Keep track of the best solution found
- Terminate when optimal solution is found or iteration limit reached

**Parameters:**
- Maximum iterations: 10,000,000
- No heuristics or learning—pure random sampling

This baseline helps us understand how much benefit comes from intelligent search strategies versus random exploration.

In [ ]:
def random_search(A, b, max_iter=10_000_000, seed=0):
    """
    Simple random search baseline.

    Parameters
    ----------
    A : np.ndarray
        Coefficient matrix
    b : np.ndarray
        Target vector
    max_iter : int
        Maximum iterations (default: 10,000,000)
    seed : int
        Random seed for reproducibility

    Returns
    -------
    tuple
        (best_solution, execution_time)
    """
    start_time = time.time()
    np.random.seed(seed)

    best_solution = None
    best_cost = float("inf")

    for i in range(max_iter):
        # Generate random binary solution
        solution = np.random.randint(0, 2, A.shape[1])
        cost = np.sum((A @ solution - b) ** 2)

        # Update best if improved
        if cost < best_cost:
            best_cost = cost
            best_solution = solution.copy()

        # Early termination if optimal found
        if best_cost == 0:
            break

    return best_solution.tolist(), time.time() - start_time


# Run Random Search
print("Running Random Search...")
random_solution, random_time = random_search(A, b)
comparator.add_result("Random Search", random_solution, random_time)

#### Baseline 4: Brute Force Search

Brute Force exhaustively enumerates all possible solutions, guaranteeing the optimal solution is found. For our 20-variable problem, this means checking all $2^{20} = 1,048,576$ possible market assignments.

**How It Works:**
- Enumerate all binary vectors of length $n$ by counting from 0 to $2^n - 1$
- Convert each integer to its binary representation
- Evaluate the cost function for each solution
- Return the solution with minimum cost

**Scalability Note:**
While feasible for 20 variables, brute force becomes intractable for larger problems:
- 30 variables: $2^{30} \approx 1$ billion solutions
- 40 variables: $2^{40} \approx 1$ trillion solutions
- 50 variables: $2^{50} \approx 1$ quadrillion solutions

This exponential growth is why quantum and classical heuristic methods are essential for real-world problem sizes.

In [ ]:
def brute_force(A, b):
    """
    Exhaustive brute force search - guaranteed to find optimal solution.

    Parameters
    ----------
    A : np.ndarray
        Coefficient matrix
    b : np.ndarray
        Target vector

    Returns
    -------
    tuple
        (best_solution, execution_time)
    """
    start_time = time.time()
    n = A.shape[1]
    best_solution = None
    best_cost = float("inf")

    # Enumerate all 2^n possible binary solutions
    for i in range(2**n):
        # Convert integer to binary solution
        solution = [(i >> j) & 1 for j in range(n)]
        cost = np.sum((A @ solution - b) ** 2)

        # Update best if improved
        if cost < best_cost:
            best_cost = cost
            best_solution = solution.copy()

        # Early termination if optimal found
        if best_cost == 0:
            break

    return best_solution, time.time() - start_time


# Run Brute Force Search
print("Running Brute Force Search...")
bf_solution, bf_time = brute_force(A, b)
comparator.add_result("Brute Force", bf_solution, bf_time)

#### Comparison Results and Analysis

Now let's visualize and analyze the results from all four approaches. We'll compare them on two key dimensions:

1. **Solution Quality**: How well does each method satisfy the constraints? (Lower objective value = better)
2. **Execution Time**: How long does each method take to find its solution?

The comparison will help us understand the trade-offs between solution quality, execution time, and algorithmic complexity for this problem size.

In [ ]:
# Print comparison summary
print("\nALGORITHM COMPARISON RESULTS")
print("=" * 70)
print(
    f"{'Algorithm':<30} | {'Objective':>10} | {'Feasible':>10} | {'Time (s)':>10}"
)
print("-" * 70)

for result in comparator.results:
    feasible_icon = "✓" if result["Feasible"] else "✗"
    print(
        f"{result['Algorithm']:<30} | {result['Objective']:>10.0f} | {feasible_icon:>10} | {result['Time']:>10.3f}"
    )

# Find best and fastest
best_result = min(comparator.results, key=lambda x: x["Objective"])
fastest_result = min(comparator.results, key=lambda x: x["Time"])

print(f"\n{'='*70}")
print(
    f"Best Solution Quality: {best_result['Algorithm']} (Objective: {best_result['Objective']})"
)
print(
    f"Fastest Execution: {fastest_result['Algorithm']} ({fastest_result['Time']:.3f}s)"
)

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

algorithms = [r["Algorithm"] for r in comparator.results]
objectives = [r["Objective"] for r in comparator.results]
times = [r["Time"] for r in comparator.results]

# Color schemes
colors_quality = ["#2E8B57" if obj == 0 else "#DC143C" for obj in objectives]
colors_time = ["#4169E1", "#FF6347", "#32CD32", "#FFD700"][: len(algorithms)]

# Plot 1: Solution Quality
bars1 = ax1.bar(
    algorithms,
    objectives,
    color=colors_quality,
    alpha=0.8,
    edgecolor="black",
    linewidth=1.2,
)
ax1.set_title(
    "Solution Quality Comparison", fontsize=14, fontweight="bold", pad=20
)
ax1.set_ylabel(
    "Constraint Violations (Lower = Better)", fontsize=12, fontweight="bold"
)
ax1.tick_params(axis="x", rotation=45, labelsize=10)
ax1.tick_params(axis="y", labelsize=10)
ax1.set_ylim(-0.1, max(objectives) * 1.2 if max(objectives) > 0 else 1)
ax1.grid(axis="y", alpha=0.3, linestyle="--")

# Add value labels on bars
for bar, obj in zip(bars1, objectives):
    height = bar.get_height()
    ax1.text(
        bar.get_x() + bar.get_width() / 2.0,
        height + max(objectives) * 0.02 if max(objectives) > 0 else 0.05,
        f"{obj:.0f}",
        ha="center",
        va="bottom",
        fontweight="bold",
        fontsize=10,
    )

# Add vertical dashed lines between bars
for i in range(len(algorithms) - 1):
    ax1.axvline(
        x=i + 0.5, color="gray", linestyle="--", alpha=0.6, linewidth=1
    )

# Plot 2: Execution Time
bars2 = ax2.bar(
    algorithms,
    times,
    color=colors_time,
    alpha=0.8,
    edgecolor="black",
    linewidth=1.2,
)
ax2.set_title(
    "Execution Time Comparison", fontsize=14, fontweight="bold", pad=20
)
ax2.set_ylabel("Time (seconds)", fontsize=12, fontweight="bold")
ax2.tick_params(axis="x", rotation=45, labelsize=10)
ax2.tick_params(axis="y", labelsize=10)
ax2.grid(axis="y", alpha=0.3, linestyle="--")
ax2.set_ylim(0, max(times) * 1.2)

# Add value labels on bars
for bar, time_val in zip(bars2, times):
    height = bar.get_height()
    ax2.text(
        bar.get_x() + bar.get_width() / 2.0,
        height + max(times) * 0.02,
        f"{time_val:.2f}s",
        ha="center",
        va="bottom",
        fontweight="bold",
        fontsize=10,
    )

# Add vertical dashed lines between bars
for i in range(len(algorithms) - 1):
    ax2.axvline(
        x=i + 0.5, color="gray", linestyle="--", alpha=0.6, linewidth=1
    )

# Styling
plt.tight_layout()
plt.subplots_adjust(bottom=0.2)
fig.patch.set_facecolor("#f8f9fa")
ax1.set_facecolor("#ffffff")
ax2.set_facecolor("#ffffff")

plt.show()

#### Interpreting the Comparison Results

**Key Observations:**

1. **Solution Quality (Constraint Violations)**:
   - **Green bars (value = 0)**: Algorithms that found perfect solutions satisfying all constraints
   - **Red bars (value > 0)**: Algorithms with constraint violations
   - For this instance, multiple algorithms can find optimal solutions, demonstrating that the problem is solvable at this scale

2. **Execution Time Trade-offs**:
   - **Brute Force**: Guaranteed optimal but time increases exponentially with problem size ($O(2^n)$)
   - **Simulated Annealing**: Often finds good solutions with reasonable time, but no guarantees
   - **Random Search**: Highly variable performance; may require many samples to find optimal solutions
   - **Quantum (bf-DCQO)**: Execution time includes quantum circuit preparation, hardware queue time, and post-processing

3. **Scalability Implications**:
   - For 20 variables: All methods can potentially succeed
   - For 30+ variables: Brute force becomes impractical ($2^{30} \approx 1$ billion evaluations)
   - For 50+ variables: Only quantum and sophisticated heuristics remain viable
   - For 100+ variables: Quantum optimization with efficient algorithms like bf-DCQO offers the most promising path

4. **When to Use Each Approach**:
   - **Brute Force**: Only for small problems (< 25 variables) where guaranteed optimality is required
   - **Simulated Annealing**: Medium-sized problems where good solutions are acceptable and classical resources are limited
   - **Random Search**: Baseline for comparison or when solutions are abundant in the search space
   - **Quantum (bf-DCQO)**: Large-scale problems (up to 156 qubits) where classical methods struggle, especially when high-quality solutions are needed and quantum hardware is accessible

**The Quantum Advantage Horizon:**

While classical methods may perform competitively at small scales like this 20-variable problem, quantum optimization shines as problems grow larger. The bf-DCQO algorithm's efficiency (10× fewer gates than other quantum approaches) positions it uniquely for the NISQ era, handling problem sizes that challenge both classical solvers and less efficient quantum algorithms.

## Conclusion

### What We Accomplished

In this tutorial, we successfully:

1. **Loaded a Real Optimization Problem**: Obtained a challenging Market Split instance from the QOBLIB benchmark library [2]
2. **Transformed to QUBO Format**: Converted the constrained problem into an unconstrained quadratic formulation [3]
3. **Leveraged Advanced Quantum Algorithms**: Used Kipu Quantum's bf-DCQO algorithm with counterdiabatic terms [1]
4. **Obtained Optimal Solutions**: Found feasible solutions satisfying all constraints

### Key Takeaways

**Algorithm Innovation**: The bf-DCQO algorithm represents a significant advancement [1]:
- **10 times fewer gates** than Digital Quantum Annealing
- **Approximately 10 iterations** instead of approximately 100 for variational methods
- **Built-in error resilience** through circuit efficiency

**Counterdiabatic Terms**: Enable rapid quantum evolution while maintaining ground state fidelity, making quantum optimization practical on today's noisy hardware [1].

**Bias-Field Guidance**: The iterative bias-field approach allows each iteration to start near previously found good solutions, providing a form of quantum-enhanced local search [1].

**Practical Quantum Advantage**: For problems like Market Split that challenge classical solvers at small scales [4], bf-DCQO provides a viable path forward.

### Next Steps

To deepen your understanding and explore further:

1. **Try Different Instances**: Experiment with other QOBLIB instances of varying sizes
2. **Tune Parameters**: Adjust `num_iterations`, `preprocessing_level`, `postprocessing_level`
3. **Compare with Classical**: Benchmark against classical optimization solvers
4. **Try different strategies**: Try to find a better encoding for the problem or formulate it as HUBO (if possible)
5. **Apply to Your Domain**: Adapt the QUBO/HUBO formulation techniques to your own optimization problems

### References

[1] IBM Quantum. "Kipu Quantum Optimization." *IBM Quantum Documentation*. https://quantum.cloud.ibm.com/docs/en/guides/kipu-optimization

[2] QOBLIB - Quantum Optimization Benchmarking Library. Zuse Institute Berlin (ZIB). https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library

[3] Glover, F., Kochenberger, G., & Du, Y. (2019). "Quantum bridge analytics I: a tutorial on formulating and using QUBO models." *4OR: A Quarterly Journal of Operations Research*, 17(4), 335-371.

[4] Lodi, A., Tramontani, A., & Weninger, K. (2023). "The Intractable Decathlon: Benchmarking Hard Combinatorial Problems." *INFORMS Journal on Computing*.